In [ ]:
import numpy as np
import pandas as pd
from ConfigSpace import ConfigurationSpace, Configuration
from smac import MultiFidelityFacade, Scenario
from functools import partial


from datasets import get_hits
from optimize import CONFIG_DEFAULTS
from metrics.tracks import track_metrics
from segment.track import gen_seg_track_layered
from segment.candidate import gen_seg_layered
from hopfield.energy.cross import cross_energy_matrix
from hopfield.energy.curvature import segment_adjacent_pairs, curvature_energy_matrix
from hopfield.iterate import annealing_curve, anneal, update_act_bulk


## 1.Learning rate is *critical*:
- small easy dataset
- many trials are needed to find a workable config
- high t_max helps
- learning rate results are still better and much faster

In [ ]:
N_TRIALS = 1000
N_EVENTS = 50
EVENT_SIZE = 10
N_NOISE_HITS = 10
N_WORKERS = -1

In [ ]:
events = get_hits('spdsim', n_events=N_EVENTS, n_noise_hits=N_NOISE_HITS, event_size=EVENT_SIZE)

events = {eid: event.reset_index(drop=True) for eid, event in events.groupby('event_id')}
eids = tuple(events.keys())

def evaluate(config: Configuration, seed: int, budget: float = 10) -> float:
    conf = CONFIG_DEFAULTS.copy()
    conf.update(config)
    rng = np.random.default_rng(seed=seed)
    scores = []
    for eid in rng.choice(eids, int(budget), replace=False):
        event = events[eid]
        seg = gen_seg_layered(event)
        crossing_matrix = conf['alpha'] * cross_energy_matrix(seg)
        curvature_matrix = curvature_energy_matrix(
            pos=event[['x', 'y', 'z']].to_numpy(),
            seg=seg, pairs=segment_adjacent_pairs(seg), alpha=conf['alpha'], gamma=conf['gamma'],
            cosine_threshold=conf['cosine_min_rewarded'], cosine_min_allowed=conf['cosine_min_allowed'],
            curvature_cosine_power=conf['cosine_power'],
            do_sum_r=conf['distance_op'] == 'sum', distance_prod_power_in_denominator=conf['distance_power']
        )
        energy_matrix = crossing_matrix + curvature_matrix
        temp_curve = annealing_curve(conf['t_min'], conf['t_max'],
                                     conf['cooling_steps'], conf['rest_steps'])
        act = np.full(len(seg), conf['initial_act'])
        update_act = partial(update_act_bulk, learning_rate=conf['learning_rate'])
        for _ in anneal(energy_matrix, temp_curve, act, update_act, bias=conf['bias']):
            pass
        tseg = gen_seg_track_layered(event)
        score = track_metrics(event, seg, tseg, act, act>0.5)
        score['total steps'] = config['cooling_steps'] + config['rest_steps']
        score['trackml loss'] = 1. - score['trackml score']
        scores.append(score)
    return pd.DataFrame(scores).mean()['trackml loss']


In [ ]:
scenario = Scenario(
    ConfigurationSpace({
        'alpha': (0., 1000.),
        'gamma': (0., 2000.),
        'bias': (-40.0, 40.0),
        'cosine_power': (0.0, 50.0),
        'cosine_min_rewarded': (0., 1.),
        't_max': (1., 500.),
        'cooling_steps': (1, 50),
        'rest_steps': (1, 10),
        'initial_act': (0., 1.),
    }),
    'easy-classic',
    n_trials=N_TRIALS,
    n_workers=N_WORKERS,
    min_budget=2,
    max_budget=N_EVENTS
)

optimizer = MultiFidelityFacade(scenario, evaluate, overwrite=True)
best_config = optimizer.optimize()
optimizer.validate(best_config)

In [ ]:
best_config

In [ ]:
pd.DataFrame(optimizer.intensifier.trajectory)

In [ ]:
scenario = Scenario(
    ConfigurationSpace({
        'alpha': (0., 1000.),
        'gamma': (0., 2000.),
        'bias': (-40.0, 40.0),
        'cosine_power': (0.0, 50.0),
        'cosine_min_rewarded': (0., 1.),
        't_max': (1., 500.),
        'cooling_steps': (1, 50),
        'rest_steps': (1, 10),
        'initial_act': (0., 1.),
        'learning_rate': (0., 1.)
    }),
    'easy-rate',
    n_trials=N_TRIALS/4,
    n_workers=N_WORKERS,
    min_budget=2,
    max_budget=N_EVENTS
)

optimizer = MultiFidelityFacade(scenario, evaluate, overwrite=True)
best_config = optimizer.optimize()
optimizer.validate(best_config)

In [ ]:
best_config

In [ ]:
pd.DataFrame(optimizer.intensifier.trajectory)

## 2. Harder problem. TODO: result? 

In [ ]:
N_TRIALS = 200
N_EVENTS = 20
EVENT_SIZE = 10
N_NOISE_HITS = 35*40
N_WORKERS = -1

In [ ]:
events = get_hits('spdsim', n_events=N_EVENTS, n_noise_hits=N_NOISE_HITS, event_size=EVENT_SIZE)

events = {eid: event.reset_index(drop=True) for eid, event in events.groupby('event_id')}
eids = tuple(events.keys())

def evaluate(config: Configuration, seed: int, budget: float = 10) -> float:
    conf = CONFIG_DEFAULTS.copy()
    conf.update(config)
    rng = np.random.default_rng(seed=seed)
    scores = []
    for eid in rng.choice(eids, int(budget), replace=False):
        event = events[eid]
        seg = gen_seg_layered(event)
        crossing_matrix = conf['alpha'] * cross_energy_matrix(seg)
        curvature_matrix = curvature_energy_matrix(
            pos=event[['x', 'y', 'z']].to_numpy(),
            seg=seg, pairs=segment_adjacent_pairs(seg), alpha=conf['alpha'], gamma=conf['gamma'],
            cosine_threshold=conf['cosine_min_rewarded'], cosine_min_allowed=conf['cosine_min_allowed'],
            curvature_cosine_power=conf['cosine_power'],
            do_sum_r=conf['distance_op'] == 'sum', distance_prod_power_in_denominator=conf['distance_power']
        )
        energy_matrix = crossing_matrix + curvature_matrix
        temp_curve = annealing_curve(conf['t_min'], conf['t_max'],
                                     conf['cooling_steps'], conf['rest_steps'])
        act = np.full(len(seg), conf['initial_act'])
        update_act = partial(update_act_bulk, learning_rate=conf['learning_rate'])
        for _ in anneal(energy_matrix, temp_curve, act, update_act, bias=conf['bias']):
            pass
        tseg = gen_seg_track_layered(event)
        score = track_metrics(event, seg, tseg, act, act>0.5)
        score['total steps'] = config['cooling_steps'] + config['rest_steps']
        score['trackml loss'] = 1. - score['trackml score']
        scores.append(score)
    return pd.DataFrame(scores).mean()['trackml loss']


In [ ]:
scenario = Scenario(
    ConfigurationSpace({
        'alpha': (0., 100.),
        'gamma': (0., 1000.),
        'bias': (-40.0, 40.0),
        'cosine_power': (0.0, 50.0),
        'cosine_min_rewarded': (0., 1.),
        't_max': (1., 500.),
        'cooling_steps': (1, 50),
        'rest_steps': (1, 10),
        'initial_act': (0., 1.),
        'learning_rate': (0., 1.)
    }),
    'basic',
    n_trials=N_TRIALS,
    n_workers=N_WORKERS,
    min_budget=2,
    max_budget=N_EVENTS
)

optimizer = MultiFidelityFacade(scenario, evaluate, overwrite=True)
best_config = optimizer.optimize()
optimizer.validate(best_config)

In [ ]:
best_config

In [ ]:
pd.DataFrame(optimizer.intensifier.trajectory)